In [1]:
import numpy as np

# Applying Kullback Leibler

### Information entropy from probability distributions

Recall from [1] and [2] the form of information entropy for a random variable $X$ with distribution $P(x)$ over a (measurable) space $x \in \mathcal{X}$

$ I(P) = \log_2 \left( \frac{1}{P(X)} \right) $,

$ H(P) = - \sum_{x \in \mathcal{X}} P(x) \log_2 P(x) $

or more generally

$ H(P) = \mathbb{E}_{X\sim P} [ - \log_2 P(X) ] = \mathbb{E}_{X\sim P} [ I(X) ]$ 

$H(P)$ represents the 'uncertainty' of $X$, or the rate of generating information from an information source $X$. $I(P)$ represents the information content of an event from $X$. In the context of messaging, $I(P)$ represents the minimum length of the message (in bits) to transmit the information content of the message, and $H(P)$ is the expected length of a message.

For a continuous random variable $X$ with probability density function $p(x)$ over a continuous space $x \in \mathcal{X}$, one can use differential entropy [4], which is NOT a $n\to\infty$ limit of information entropy 

$ H(p) = \int_\mathcal{X} dx p(x) \log_2 p(x) $

### Cross entropy from probability distributions

The above definitions take into account only a single distribution over $X$. In practice, there may be multiple distributions over $X$, in particular an estimate (given a certain amount of evidence) of the distribution over $X$. Cross entropy allows extending the definitions of entropy to multiple distributions over the same random variable.

Suppose now that messages/information content are generated by a sender obeying another distribution $Q(x)$. One then has

$ I(Q) = \log_2 \left( \frac{1}{Q(X)} \right) $

However, the true distribution is $P(X)$. This means that the expected message length to correctly convey the information is

$ H(P,Q) := \mathbb{E}_{X\sim P} [ I(Q) ] = - \sum_{x \in \mathcal{X}} P(x) \log_2 Q(x) $

The cross entropy is useful for considering the discrepancy between a true distribution, $P(X)$, and some candidate or estimated distribution $Q(X)$ that one might be working with. It represents the 'uncertainty' of $X$ when one is using a presumed distribution $Q(X)$. Some good exploratory examples are available here [5]. The extra uncertainty that arises over $H(P)$ can be used to define a quantity; namely the Kullback-Leibler divergence

$ D_{KL}(P||Q) := H(P,Q) - H(P) $

By Gibbs' inequality [6], one finds that $ D_{KL}(P||Q) \geq 0 $ and $ D_{KL}(P||Q) = 0 $ if and only if $ P = Q $. $D_{KL}$ gives some kind of distance of how far $Q$ is from $P$, although does not form a distance metric, but does form a statistical divergence [7].

In [7]:
def kl_div(p, q):
    return  - np.sum(np.array(p) * np.log2(q)) + np.sum(np.array(p) * np.log2(p))

In [15]:
n_sample = 20
p = np.random.random(n_sample)
q = np.random.random(n_sample)
print(f"KL divergence of a distribution from itself = {kl_div(p, p)}")
print(f"KL divergence of a distribution from another = {kl_div(p, q)}")

KL divergence of a distribution from itself = 0.0
KL divergence of a distribution from another = 3.5427017090192026


### References

[1] Shannon, C. ["A mathematical theory of communication"](https://people.math.harvard.edu/~ctm/home/text/others/shannon/entropy/entropy.pdf)

[2] Wikipedia, ["Entropy"](https://en.wikipedia.org/wiki/Entropy_(information_theory))

[3] Wikipedia, ["Conditional Entropy"](https://en.wikipedia.org/wiki/Conditional_entropy)

[4] Wikipedia, ["Differential Entropy"](https://en.wikipedia.org/wiki/Differential_entropy)

[5] Towards Data Science, ["Entropy, Cross-Entropy, and KL-Divergence Explained!"](https://towardsdatascience.com/entropy-cross-entropy-and-kl-divergence-explained-b09cdae917a)

[6] Wikipedia, ["Gibbs' inequality"](https://en.wikipedia.org/wiki/Gibbs%27_inequality#Proof)

[7] Wikipedia, ["Divergence_(statistics)"](https://en.wikipedia.org/wiki/Divergence_(statistics))